# Рекомендация тарифов

Откроем файл с данными и изучим их, для этого подключим библиотеку `pandas` помимо него подключим еще остальные библиотеки который нам пригодятся. Для того чтобы прочитать данные из датасета воспользуемся методом `read_csv`, для получения общей информации о датасете воспользуемся методом `info`.

### Общая информация

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
try:
    data = pd.read_csv('/datasets/users_behavior.csv')
except:
    data = pd.read_csv('users_behavior.csv')
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


По данным видно, что у нас есть 5 различных столбцов с определнной информацией, у всех столбцов опреден числовой тип данных и пропусков в столбцах нет. Проверим есть ли в них явные дубликаты.

In [4]:
data.duplicated().sum()

0

Явных дубликатов нет. Если посмотреть на первые 5 строк из датасета, то так как там указны только числа, можно прийти к выводу, что неявных дубликатов нет в датасете.

### Разделение данных на выборки

Данные заранее были уже обработаны, предобработка не нужна, можно переходить к разделение данных на выборки для обучения моделей машинного обучения. Разобьем данные на три выборки — обучащая **60%**, валидационная **20%** и тестовая **20%**, для этого воспользуемся методом `train_test_split` из библиотеки `sklearn`.

In [5]:
#сначала разбиваем выборку на обучающую и валидационную, затем из валидационной выборки получаем тестовую
data_train, data_valid = train_test_split(data, test_size=0.4, random_state=12345)
data_valid, data_test = train_test_split(data_valid, test_size=0.5, random_state=12345)
print(data_train.shape)
print(data_valid.shape)
print(data_test.shape)

(1928, 5)
(643, 5)
(643, 5)


По размерности видно, что данные разделились корректно, выбирем теперь из данных признаки и что необходимо предсказать. В задание указано, что нужно **построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра»**, каким тарифом пользовался клиент указано в столбце `is_ultra` («Ультра» — 1, «Смарт» — 0), в остальных столбцах указана информация о том сколько было звонков, потраченных минут, количестов sms-сообщений и зрасходованный интернет-трафик в Мб, эту информацию можно взять за признаки по которыму можно предсказать какой тариф пождходит клиенту.

В качестве признаков возьмем столбцы `calls`, `minutes`, `messages`, `mb_used` и обучим модель, чтобы она помогла предсказать значение в столбце `is_ultra`.

In [6]:
data_train_features = data_train.drop(['is_ultra'], axis=1)
data_train_target = data_train['is_ultra']

data_valid_features = data_valid.drop(['is_ultra'], axis=1)
data_valid_target = data_valid['is_ultra']

data_test_features = data_test.drop(['is_ultra'], axis=1)
data_test_target = data_test['is_ultra']

### Исследование моделей

Возьмем для решения нашей задачи несколько моделей машинного обучения — **Дерево решений**, **Случайный лес**, **Логистическая регрессия** и переберем в них гиперпараметры, чтобы найти лучшую модель для решения нашей задачи. Для оценок модели будем использовать метрику качества *accuracy*.

Начнем с модели **Дерево решений**:

In [7]:
best_model_tree = None
best_result_model_tree = 0

for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(data_train_features, data_train_target)
    predictions_tree = model_tree.predict(data_valid_features)
    result_tree = accuracy_score(data_valid_target, predictions_tree)
    
    if result_tree > best_result_model_tree:
        best_result_model_tree = result_tree
        best_model_tree = model_tree
    
    print('Глубина дерева:', depth, 'Точность модели:', result_tree)

Глубина дерева: 1 Точность модели: 0.7542768273716952
Глубина дерева: 2 Точность модели: 0.7822706065318819
Глубина дерева: 3 Точность модели: 0.7853810264385692
Глубина дерева: 4 Точность модели: 0.7791601866251944
Глубина дерева: 5 Точность модели: 0.7791601866251944
Глубина дерева: 6 Точность модели: 0.7838258164852255
Глубина дерева: 7 Точность модели: 0.7822706065318819
Глубина дерева: 8 Точность модели: 0.7791601866251944
Глубина дерева: 9 Точность модели: 0.7822706065318819
Глубина дерева: 10 Точность модели: 0.7744945567651633


Модель **Дерево решений** на валидационной выборке показывает точность **75%** с максимальной глубиной дерева равной **1**, далее с увелечением числа максимальной глубины дерева точность немного вырастает до **78.5%** и начиная с максимальной глубины дерева равной **3**, точность модели **Дерево решений** немного падает, она конечно не опускается до **75%**, но максимального значения точности **78.5%** больше не достигает.

Обучим далее модель **Случайного леса**, может у неё получится получится точность выше, чем у модели **Дерево решений**. У модели **Случайного леса** будем перебирать два гиперпараметра, `max_depth` — максимальная глубина дерева и `n_estimators` — количество деревьев в лесе, для поиска более оптимального решения:

In [8]:
best_model_randomforest = None
best_result_randomforest = 0

for depth in range(1, 11, 2):
    for est in range(10, 51, 10):
        model_randomforest = RandomForestClassifier(max_depth=depth, n_estimators=est)
        model_randomforest.fit(data_train_features, data_train_target)
        predictions_randomforest = model_randomforest.predict(data_valid_features)
        result_randomforest = accuracy_score(data_valid_target, predictions_randomforest)
        
        if result_randomforest > best_result_randomforest:
            best_result_randomforest = result_randomforest
            best_model_randomforest = model_randomforest
            
        print('Количество деревьев в лесе:', est, 'Глубина дерева:', depth, 'Точность модели:', result_randomforest)

Количество деревьев в лесе: 10 Глубина дерева: 1 Точность модели: 0.7884914463452566
Количество деревьев в лесе: 20 Глубина дерева: 1 Точность модели: 0.749611197511664
Количество деревьев в лесе: 30 Глубина дерева: 1 Точность модели: 0.7869362363919129
Количество деревьев в лесе: 40 Глубина дерева: 1 Точность модели: 0.7558320373250389
Количество деревьев в лесе: 50 Глубина дерева: 1 Точность модели: 0.7542768273716952
Количество деревьев в лесе: 10 Глубина дерева: 3 Точность модели: 0.7916018662519441
Количество деревьев в лесе: 20 Глубина дерева: 3 Точность модели: 0.7884914463452566
Количество деревьев в лесе: 30 Глубина дерева: 3 Точность модели: 0.7900466562986003
Количество деревьев в лесе: 40 Глубина дерева: 3 Точность модели: 0.7900466562986003
Количество деревьев в лесе: 50 Глубина дерева: 3 Точность модели: 0.7869362363919129
Количество деревьев в лесе: 10 Глубина дерева: 5 Точность модели: 0.7853810264385692
Количество деревьев в лесе: 20 Глубина дерева: 5 Точность модели: 

По сравнению с моделью **Дерево решений** модель **Случайного леса** показывает результат точности на валидационной выборке лучше - максимальная точность достигает **80%**, при количестве деревьев равным **40** и максимальной глубине дерева равной **7**, далее с увеличением значений в гиперпараметрах точность модели **Случайного леса** немного падает, она конечно не опускается до **75%**, но максимального значения точности **80%** больше не достигает.

Проверим ещё модель **Логистической регрессии**, может обучив её, мы получим точность выше:

In [9]:
model_logistic = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_logistic.fit(data_train_features, data_train_target)
predictions_logistic = model_logistic.predict(data_valid_features)
result_logistic = accuracy_score(data_valid_target, predictions_logistic)
        
print('Точность модели:', result_logistic)

Точность модели: 0.7107309486780715


Модель **Логистической регрессии** показала точность равную **71%** самый низкий результат по сравнению с моделями **Дерево решений** — точность **78.5%** и **Случайный лес** — точность **80%**. На основании выше проведенного исследования мы нашли модель машинного обучения, которую можно использовать на тествой выборке — это модель **Случайный лес** с гиперпараметрами `n_estimators` — количество деревьев в лесу равное **40** и `max_depth` — глубина дерева равное **7**.

### Проверка на тестовой выборке

Ранее нам удалось определить, что лучшую точность при обучении показала модель **Случайный лес** с гиперпараметрами `n_estimators` — количество деревьев в лесу равное **40** и `max_depth` — глубина дерева равное **7**. Проверим теперь выбранную нами модель на тестовых данных:

In [10]:
model_randomforest = RandomForestClassifier(random_state=12345, max_depth=7, n_estimators=40)
model_randomforest.fit(data_train_features, data_train_target)
predictions_randomforest = model_randomforest.predict(data_test_features)
result_randomforest = accuracy_score(data_test_target, predictions_randomforest)
print('Точность модели "Случайный лес" на тестовой выборке', result_randomforest)

Точность модели "Случайный лес" на тестовой выборке 0.80248833592535


Как можно заметить у модели **Случайный лес** на тестовой выборке точность не изменилась — осталась на уровне **80%**, что говорит о том, что мы сделали правильный выбор в пользу этой модели.

### Проверка моделей на вменяемость

Для проверки моделей на вменяемость воспользуемся моделью `DummyClassifier` из библиотеки `sklearn` эта модель используется для проверки более сложных моделей. Чтобы проверить с помощью данной модели обученную модель **Случайного леса** укажем для гиперпараметра `strategy` — *стратегия, используемая для создания прогнозов* значение `uniform`, тогда модель `DummyClassifier` сгенерирует прогнозы равномерно случайным образом из списка уникальных классов, то есть в модели равновероятно будут распределены значения выбор тарифа «Ультра» — 1 и «Смарт» — 0:

In [11]:
model_dummy = DummyClassifier(strategy='uniform')
model_dummy.fit(data_train_features, data_train_target)
predictions_dummy = model_dummy.predict(data_test_features)
result_dummy = accuracy_score(data_test_target, predictions_dummy)
print('Точность случайной модели на тестовой выборке', result_dummy)

Точность случайной модели на тестовой выборке 0.5225505443234837


Точность модели, с равномерным случайным распределением значений выбора тарифа, на тестовой выборке находится рядом со значением **50%**, что говорит о вменяемости наших обученных модели **Случайного леса** у которой точность равна **80%**.

### Общий вывод

В рамках задания нам нужно было построить модель машинного обучения, которая смогла бы рекомендовать новый тариф пользователю на основании использования мобильной связи, точность обуения модели должна быть составлять не менее **75%**. 

Было обучено несколько моделей машинного обучения по предоставленным данным и получены следующие результаты:
- Модель **Дерево решений** точность на валидационной выборке — **78.5%**;
- Модель **Случайный лес** точность на валидационной выборке — **80%**;
- Модель **Логистическая регрессия** точность на валидационной выорке — **71%**.

На основании проведенной проверки обучения моделей на валидационной выборке была выбрана модель **Случайный лес** для проверки на тестовой выборке, на тестовой выборке данная модель показала такую же точность как и на валидационной выборке и составила **80%**.

На основании выше перечисленного для рекомендации тарифа пользователю необходимо использовать модель **Случайный лес**, так как данная модель показала максимальную точность **80%** и на тестовой выборке эта точность не изменилась.